In [1]:
import pandas as pd
import shutil
import zipfile
import os

from pathlib     import Path
from collections import defaultdict

In [2]:
types = {
    0: "Bass",
    1: "Brass",
    2: "Keys",
    3: "Lead",
    4: "Organ",
    5: "Pad",
    6: "Percussion",
    7: "Sequence",
    8: "SFX",
    9: "Strings"
}

In [3]:
types_short = {
    0: "BS",
    1: "BR",
    2: "KS",
    3: "LD",
    4: "OR",
    5: "PD",
    6: "DR",
    7: "SQ",
    8: "FX",
    9: "ST"
}

In [4]:
clean_dir     = Path("../patches/clean")
generated_dir = Path("../patches/generated")
temp_dir      = Path("../patches/temp")

generated_dir.mkdir(parents=True, exist_ok=True)

In [5]:
patches = []

for clean_file in clean_dir.glob("*.csv"):

    patch = pd.read_csv(clean_file)
    patch["bank_name"] = clean_file.stem

    patch = patch[[
        "bank_name",
        "patch_name",
        "patch_type",
        "path"
    ]]

    patch = patch[patch["patch_name"] != "Init"]

    patches.append(patch)

In [6]:
patches = pd.concat(patches)
patches = patches.drop_duplicates(subset=["patch_name", "patch_type"])
patches = patches.reset_index(drop=True)

In [7]:
patches["patch_type_short"]  = patches["patch_type"].map(types_short)
patches["patch_type"]        = patches["patch_type"].map(types)

In [8]:
patches[patches.patch_type == "Organ"]

,bank_name,patch_name,patch_type,path,patch_type_short
323,extra_tech_loop,A is,Organ,extra_tech_loop/Tech Loop/01-Tech Loop 0828 15...,OR
365,extra_future_rave,Duorgan,Organ,extra_future_rave/Future Rave/01-JB Rave bank ...,OR
415,extra_voltage_forms,Dub Ride,Organ,extra_voltage_forms/Voltage Forms/01-Voltage F...,OR
566,factory_3,Cristal Church,Organ,factory_3/Factory3.0/01-00122021 17h55-A/30-00...,OR
575,factory_3,Organism,Organ,factory_3/Factory3.0/01-00122021 17h55-A/120-0...,OR
619,factory_3,Shy Robot,Organ,factory_3/Factory3.0/01-00122021 17h55-A/304-0...,OR
705,factory_3,Jobs,Organ,factory_3/Factory3.0/01-00122021 17h55-A/20-00...,OR
803,extra_trancentral,Luv,Organ,extra_trancentral/Trancentral/01-09202022 10h0...,OR
862,factory_5,Rhody,Organ,factory_5/New Presets 5.0/01-Factory 5.0 .97-A...,OR
885,factory_5,Hartonium,Organ,factory_5/New Presets 5.0/01-Factory 5.0 .97-A...,OR


In [9]:
paths_by_type = defaultdict(list)

for _, row in patches.iterrows():
    patch_type = row["patch_type"]
    patch_path = row["path"]

    src_path = temp_dir / patch_path
    paths_by_type[patch_type].append(src_path)

In [10]:
for patch_type, paths in paths_by_type.items():
    type_dir = generated_dir / patch_type
    subfolder = type_dir / f"01-{patch_type}-A"
    subfolder.mkdir(parents=True, exist_ok=True)

    for i, src_path in enumerate(paths):
        patch_num = i + 1
        new_name = f"01-{patch_type}-A{patch_num}.mbp"
        dst_path = subfolder / new_name
        if src_path.exists():
            shutil.copy(src_path, dst_path)